In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Performance metrics
from sklearn.metrics import mean_absolute_error

from preprocessing import create_and_preprocess_X_y, create_train_test_set
from model import create_y_pred_baseline

In [2]:
X, y = create_and_preprocess_X_y()
X[0,:,:]

['mazrka40', 'mazrka21', 'mazrka16', 'mazrka35', 'mazrka34', 'chopin_e', 'mazrka42', 'contreda', 'skchonoc', 'chop23b', 'mazrka27', 'mazrka09', 'choschz2', 'mazrka02', 'choval13', 'mazrka15', 'prelop45', 'mazrka19', 'mazrka50', 'scherzo1', 'mazrka10', 'mazrka31', 'op72', 'ballade2', 'chv8pbi3', 'varigerm', 'chpnbarc', 'mazrka07', 'mazurka_', 'mk_chim2', 'chlargo', 'mazrka24', 'chopineb', 'berceuse', 'mazrka36', 'mazrka03', 'mazrka51', 'mazrka12', 'ballade3', 'mazurka', 'mazrka20', 'mazrka17', 'mazrka26', 'algrcrt3', 'chsc3', 'mazrka44', 'mazrka08', 'chpnimpu', 'mazrka37', 'ch_wal03', 'mazrka05', 'mazrka13', 'chonoc07', 'mazrka06', 'mazrka28', 'mazrka43', 'mazrka14', 'lecpsd3', 'mazrka38', 'chetude1', 'souvpaga', 'mazrka01', 'fugaam', 'variludo', 'mazrka32', 'mazrka18', 'chophex4', 'mazrka33', 'mazrka22', 'nouv3', 'noct15-1', 'chopnpol', 'mazrka11', 'mazrka45', 'cmazurka', 'mazrka29', 'mazrka39', 'ch_pre20', 'lecpsa3', 'op32-1', 'mazrka04', 'mazrka25', 'mazrka30', 'mazrka41', 'mazrka23'

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
X_train, X_test, y_train, y_test = create_train_test_set(X,y)

In [10]:
y_train

array([[[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]],

       ...,

       [[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.

In [9]:
print(X_train.shape, y_train.shape)

(70, 7400, 106) (70, 100, 106)


In [6]:
y_train_pred, y_test_pred = create_y_pred_baseline()

In [7]:
print(y_train_pred.shape, y_test_pred.shape)

(70, 100, 106) (18, 100, 106)


In [8]:
y_train_pred[0,:,:]

array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]])

In [1]:
metric_baseline = 1 - (abs(y_test - y_test_pred) / (y_test + y_test_pred))
metric_baseline

NameError: name 'y_test' is not defined

In [ ]:
def constant_prediction(X, y):
    errors = []
    for X, y in XXX:
        y_baseline_pred = X[:,7300:7400,:]
        
        errors.append(y - y_baseline_pred)
        
    return errors

errors_baseline = constant_prediction(X, y)
mae_baseline = np.mean(np.abs(errors_baseline))

print(mae_baseline)
